In [1]:
#!pip install tf-keras
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from transformers import Trainer, TrainingArguments
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    HfArgumentParser, 
    TrainingArguments
)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

2025-03-02 10:27:03.867084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740911223.888524 3842443 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740911223.895306 3842443 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 10:27:03.919593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from datasets import load_dataset

# Load the P3 dataset
dataset = load_dataset("openai/summarize_from_feedback", "comparisons")
print(dataset)

Reusing dataset summarize_from_feedback (/home/jupyter-st125490/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['info', 'summaries', 'choice', 'worker', 'batch', 'split', 'extra'],
        num_rows: 92858
    })
    validation: Dataset({
        features: ['info', 'summaries', 'choice', 'worker', 'batch', 'split', 'extra'],
        num_rows: 86086
    })
})


* OpenAI has made available a dataset that contains human preferences for generated text.
* This dataset is specifically focused on training reinforcement learning models with human feedback.
* It's smaller than P3 and might be more manageable.

**Preprocessing:**
**We can preprocess this dataset similarly by extracting prompts, responses, and human feedback**

In [3]:
def extract_data(sample_):
    prompt = sample_['info']['post']
    response_1 = sample_['summaries'][0]['text']
    response_2 = sample_['summaries'][1]['text']
    preference = sample_['choice']
    return { 'prompt': prompt,
            'response_1': response_1,
            'response_2': response_2,
            'preference': preference}

# small_train_dataset = processed_data.select(range(int(len() * 0.2))) 
sample = [dataset['train'][i] for i in range(0, 500)]
sample_eval = [dataset['validation'][i] for i in range(0, 10)]

In [4]:
sam = {'prompt': [], 'label':[],'preference':[]}
for s in sample:
    r = extract_data(sample_=s)
    sam['prompt'].append(r['prompt'])
    sam['label'].append([r['response_1'], r['response_2']])
    sam['preference'].append(r['preference'])


In [5]:
sam_eval  = {'prompt': [], 'label':[],'preference':[]}
for s in sample_eval:
    r = extract_data(sample_=s)
    sam_eval['prompt'].append(r['prompt'])
    sam_eval['label'].append([r['response_1'], r['response_2']])
    sam_eval['preference'].append(r['preference'])

In [6]:
import pandas as pd
x=pd.DataFrame(sam)
t = pd.DataFrame(sam_eval)
data = x.assign(label=x['label']).explode("label")

In [7]:
from transformers import BartForConditionalGeneration, BartTokenizer
model_name = "gpt2"  # or any other pre-trained model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [8]:
data.reset_index(drop=True, inplace=True)
data.head()

,prompt,label,preference
0,My boyfriend and I are long distance. We have ...,Mum is mad at me for not flying on my own tri...,1
1,My boyfriend and I are long distance. We have ...,I have made sure my mother is comfortable wit...,1
2,My boyfriend and I are long distance. We have ...,I have made sure my mother is comfortable wit...,1
3,My boyfriend and I are long distance. We have ...,mum isn't speaking to me because I booked a f...,1
4,My boyfriend and I are long distance. We have ...,mum isn't speaking to me because I booked a f...,0


In [8]:
# max_len = 512
# def preprocess_and_tokenize(batch):
#     # Extracting the prompt and responses
#     prompts = batch['info']
#     choice = batch['choice']
#     # print("***************************",batch['summaries'])
#     # print("*****************************",batch['summaries'][1])
#     # responses_1 = batch['summaries'][0]  # human response
#     # responses_2 = batch['summaries'][1]  # GPT response

#     # Initialize lists to store tokenized outputs
#     input_ids = []
#     attention_masks = []
#     labels = []
#     pref = []
#     # Loop over each example in the batch
#     for i in range(len(prompts)):
#         prompt = prompts[i].get('post', None)  # Extract 'post' from the 'info' field
#         if prompt is None or len(prompt) == 0:
#             prompt = '<|endoftext|>'
#             print(i)
#             print(batch['split'])
#         response_1 = batch['summaries'][i][0]['text']  # human-generated text
#         if response_1 is None:
#             # response_1 = '<|endoftext|>'
#             response_1 = batch['summaries'][i][0]['EDIT']
#             # print(i)
#             # print(batch['split'])
#         if len(response_1) == 0:
#             response_1 = '<|endoftext|>'
#         response_2 = batch['summaries'][i][1]['text']  # GPT-generated text
#         if response_2 is None:
#             # response_2 = '<|endoftext>'
#             response_2 = batch['summaries'][i][0]['EDIT']
#         if len(response_2) == 0:
#             response_2 = '<|endoftext|>'
#             # print(i)
#             # print(batch['split'])
#         # Tokenizing the prompt (max length = 1000)
#         # few of the last rows are null in validation set
#         prompt_encoding = tokenizer(text=prompt, truncation=True, padding="max_length", max_length=512, return_tensors = "pt")
        
#         # Tokenizing both responses (max length = 512 for responses to ensure they are short)
#         response_1_encoding = tokenizer(text=response_1, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
#         response_2_encoding = tokenizer(text=response_2, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
#         # Append tokenized results
#         input_ids.append(prompt_encoding['input_ids'].squeeze(0).tolist())
#         attention_masks.append(prompt_encoding['attention_mask'].squeeze(0).tolist())
#         labels.append([response_1_encoding['input_ids'].squeeze(0).tolist(), response_2_encoding['input_ids'].squeeze(0).tolist()])
#         # labels.append()
#         pref.append(float(choice[i]))

        
#     return {
#         'input_ids': input_ids,
#         'attention_mask': attention_masks,
#         'labels': labels,
#         'preferences': pref
#     }

# # Set pad token to eos token (this ensures padding will be handled by the tokenizer)
# tokenizer.pad_token = tokenizer.eos_token
# # Apply preprocessing to the validation dataset (with batched=True)
# processed_data = dataset['train'].map(preprocess_and_tokenize, batched=True)
# # processed_val_dataset = dataset['validation'].map(preprocess_and_tokenize, batched=True)

In [100]:
x

,prompt,label,preference
0,My boyfriend and I are long distance. We have ...,[ Mum is mad at me for not flying on my own tr...,1
1,My boyfriend and I are long distance. We have ...,[ I have made sure my mother is comfortable wi...,1
2,My boyfriend and I are long distance. We have ...,[ mum isn't speaking to me because I booked a ...,0
3,My boyfriend and I are long distance. We have ...,[ Mum thought I was going to road trip with my...,0
4,My landlord left a falsified message taped to ...,[ My landlord is harassing me and my neighbour...,1
...,...,...,...
995,Ever since I hit puberty (when I was about nin...,"[ My mom always asks me to put on ""modest"" clo...",1
996,Ever since I hit puberty (when I was about nin...,[ My mom constantly asks me to cover my body w...,0
997,Ever since I hit puberty (when I was about nin...,[ My mom always makes me cover up when around ...,1
998,Ever since I hit puberty (when I was about nin...,[ My mom repeats the same phrases about me eve...,1


In [9]:
# import pandas as pd
# pd.DataFrame(processed_data)


# Sample data (assuming it's in a pandas DataFrame)

def tokenize_data_separately(df):
    tokenized_data = []

    for _, row in df.iterrows():
        prompt = row['prompt']
        label = row['label']
        preference = row['preference']

        # Tokenize the prompt separately
        prompt_tokens = tokenizer(prompt, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

        # Tokenize the label separately
        response_1 = tokenizer(label[0], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        response_2 = tokenizer(label[1], padding="max_length", truncation=True, max_length=512, return_tensors="pt")


        tokenized_data.append({
            'input_ids': prompt_tokens['input_ids'].squeeze(),  # Prompt token IDs
            'attention_mask': prompt_tokens['attention_mask'].squeeze(),  # Prompt attention mask
            'response_1': response_1['input_ids'].squeeze(),  # Label token IDs (as target)
            'response_2': response_2['input_ids'].squeeze(),
            'preference': preference  # Reward (preference)
        })

    return tokenized_data

# Tokenize the dataset
tokenized_data_train = tokenize_data_separately(x)
tokenized_data_test = tokenize_data_separately(t)

# Process the data
# Inspect the tokenized data
# for item in tokenized_data[:2]:  # Displaying the first 2 examples
#     print(item)
from datasets import Dataset
# Convert the tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    'input_ids': [item['input_ids'] for item in tokenized_data_train],
    'attention_mask': [item['attention_mask'] for item in tokenized_data_train],
    'response_1': [item['response_1'] for item in tokenized_data_train],
    'response_2': [item['response_2'] for item in tokenized_data_train],
    'preference': [item['preference'] for item in tokenized_data_train]
})

dataset_eval = Dataset.from_dict({
    'input_ids': [item['input_ids'] for item in tokenized_data_test],
    'attention_mask': [item['attention_mask'] for item in tokenized_data_test],
    'response_1': [item['response_1'] for item in tokenized_data_test],
    'response_2': [item['response_2'] for item in tokenized_data_test],
    'preference': [item['preference'] for item in tokenized_data_test]
})
# Inspect the dataset
# print(dataset)


In [9]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'response_1', 'response_2', 'preference'],
    num_rows: 1000
})

In [178]:
dataset_eval

Dataset({
    features: ['input_ids', 'attention_mask', 'response_1', 'response_2', 'preference'],
    num_rows: 500
})

In [10]:
class DPOTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # ... (rest of the method remains the same)

        return (loss, outputs_1) if return_outputs else loss

    def log(self, logs):
        if self.state.epoch is not None:
            logs["epoch"] = self.state.epoch
        if self.state.global_step is not None:
            logs["step"] = self.state.global_step
        if self.state.max_steps is not None:
            logs["max_steps"] = self.state.max_steps

        # Add validation metrics logging
        if "eval_loss" in logs:
            self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)

    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        evaluation_outputs = super().evaluation_loop(
            dataloader,
            description=description,
            prediction_loss_only=prediction_loss_only,
            ignore_keys=ignore_keys,
            metric_key_prefix=metric_key_prefix,
        )

        # Log validation metrics
        logs = {}
        logs[f"{metric_key_prefix}_loss"] = evaluation_outputs.loss
        logs[f"{metric_key_prefix}_accuracy"] = evaluation_outputs.metrics["accuracy"]
        self.log(logs)

        return evaluation_outputs

In [15]:
import torch
from torch import nn
from time import time

class DPOTrainer(Trainer):
    # def __init__(self, *args, **kwargs):
    #     super(DPOTrainer, self).__init__(*args, **kwargs)
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract inputs
        # dict_keys(['input_ids', 'attention_mask', 'response_1', 'response_2', 'preference'])
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        labels = inputs['response_1']  # Chosen response
        labels_2 = inputs['response_2']  # Rejected response
        preferences = inputs['preference']  # Preference (1 for chosen, 0 for rejected)

        # Forward pass for the chosen response
        outputs_1 = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss_1 = outputs_1.loss

        # Forward pass for the rejected response
        outputs_2 = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_2)
        loss_2 = outputs_2.loss
        # Preferences tensor with batch size of 8


        # DPO loss calculation: minimize the loss for the chosen response
        # and maximize the loss for the rejected response
        preference_loss = (preferences * loss_1) + ((1 - preferences) * loss_2)

        # Return the combined loss
        loss = preference_loss.mean()
        
        return (loss, outputs_1) if return_outputs else loss

    # def log(self, logs, time):
    #     if self.state.epoch is not None:
    #         logs["epoch"] = self.state.epoch
    #     if self.state.global_step is not None:
    #         logs["step"] = self.state.global_step
    #     if self.state.max_steps is not None:
    #         logs["max_steps"] = self.state.max_steps

    #     # Add validation metrics logging
    #     if "eval_loss" in logs:
    #         self.control = self.callback_handler.on_log(self.args, self.state, self.control, logs)

    # def log(self,logs, start_time):
    #     if self.state.epoch is not None:
    #         logs['epoch'] = self.state.epoch
    #     if self.state.global_step is not None:
    #         logs['step'] = self.state.global_step
    #     if self.state.max_steps is not None:
    #         logs['max_steps'] = self.state.max_steps

    #     if "eval_loss" in logs:
    #         self.control = self.callback_handler.on_log(self.args, self.state ,self.control, logs)
    #     print("validation", logs)

    def evaluation_loop(
    self,
    dataloader,
    description,
    prediction_loss_only=None,
    ignore_keys=None,
    metric_key_prefix="eval",
    ):
        evaluation_outputs = super().evaluation_loop(
            dataloader,
            description=description,
            prediction_loss_only=prediction_loss_only,
            ignore_keys=ignore_keys,
            metric_key_prefix=metric_key_prefix,
        )
        
        # Calculate the loss separately
        total_loss = 0
        for batch in dataloader:
            inputs = batch
            loss = self.compute_loss(self.model, inputs)
            total_loss += loss.item()
        
        # Calculate the average loss
        average_loss = total_loss / len(dataloader)
        print("Validation_loss",average_loss)        # Log the evaluation metrics
        logs = {}
        logs[f"{metric_key_prefix}_loss"] = average_loss
        # logs[f"{metric_key_prefix}_accuracy"] = 
        # start_time = time()
        # self.log(logs, start_time)
    
        return evaluation_outputs


In [12]:
learning_rate = 1e-5
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
max_length= 512 
max_prompt_length = 128 
max_target_length =128 
label_pad_token_id = 100
max_steps = 1000
# instrumentation
sanity_check = True
report_to = None
gradient_checkpointing = None
beta = 0.1

In [13]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # Batch size for training on each device (e.g., GPU)
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    evaluation_strategy="steps",
    logging_first_step=True,
    logging_steps=100,  # match results in blog post
    eval_steps=100,
    output_dir="./test",
    optim="rmsprop",
    warmup_steps=150,
    report_to=report_to,
    bf16=True,
    gradient_checkpointing=gradient_checkpointing,
    num_train_epochs=3,  # Number of training epochs, 

    # TODO: uncomment that on the next transformers release
    # gradient_checkpointing_kwargs=gradient_checkpointing_kwargs,
)


/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
import os, gc
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
gc.collect()
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
# Set the environment variable to specify the GPUs (1, 2, and 3)
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
# PYTORCH_CUDA_ALLOC_CONF={'expandable_segments':True}
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset = dataset_eval,
    tokenizer=tokenizer,
)
dpo_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_3842443/2248676155.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  dpo_trainer = DPOTrainer(


Step,Training Loss,Validation Loss
100,0.439900,No log
200,0.391800,No log
300,0.365100,No log
400,0.349200,No log
500,0.358000,No log
600,0.337400,No log
700,0.323900,No log
800,0.311700,No log
900,0.318300,No log
1000,0.302700,No log


Validation_loss 0.5874071642756462
Validation_loss 0.5894651040434837
Validation_loss 0.5628296360373497
Validation_loss 0.5718576163053513
Validation_loss 0.5793038532137871


/home/jupyter-st125490/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Validation_loss 0.5730819553136826
Validation_loss 0.5718317925930023
Validation_loss 0.567954458296299
Validation_loss 0.5693484768271446
Validation_loss 0.5691354125738144


TrainOutput(global_step=1000, training_loss=0.34970706948637964, metrics={'train_runtime': 653.433, 'train_samples_per_second': 4.591, 'train_steps_per_second': 1.53, 'total_flos': 782569635840000.0, 'train_loss': 0.34970706948637964, 'epoch': 5.9880239520958085})

**Tried to find Validation Loss Issue but fail to do so may be inference might give more insight.**
**Validation loss is not stable due to low sample size, but keep it smaller because of resource limitation, for two summary generations it works below random**

## Inference Process in RLHF

In [33]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
# Load trained model and tokenizer


dataset_valid = load_dataset("openai/summarize_from_feedback", "comparisons")
idx =np.random.choice(len(dataset_valid['validation']))
Input_text=dataset_valid['validation'][idx]['info']['post']
dataset_valid['validation'][idx]['info']['post']

Reusing dataset summarize_from_feedback (/home/jupyter-st125490/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

"Using a throwaway since I don't want this tied to my main account. Anyways, I have a friend whom I haven't talked to in a year. I recently made a new facebook, added her (which she accepted), and sent her a message. We've known each other since high school, so about 5 years now.\n\nI asked her if she would like to meet up to catch up on things, she said she might be free this week (that was last friday). I told her just message me with a convenient time and she said sure. Now, I'm close to positive she won't respond again, which is fine, but I've been trying to mend a lot of friendships. I've 'restored' 3 or 4 now.\n\nI'm trying to change my lifestyle, and 'old me' would have sent a couple messages on monday/tuesday, but I don't want to sound creepy or needy. I genuinely want to send a thought out apology.\n\nI just need some help on getting it across in a decent way. Even if she just says she doesn't want to be my friend, I just want to send it as a peace of mind. I've restored 3 goo

In [20]:
input_text = """
My boyfriend and I are long distance. We have been trying to make it work, but we struggle with time zones, 
communication, and missing each other. We try to visit every few months, but it's always hard being apart. 
We talk every day, but sometimes I feel disconnected because of the distance. 
We've been planning for the future, and I'm hopeful that we will eventually live closer, but right now, it feels like a lot of work.
"""

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

In [46]:
import random
# # Input text for generating multiple responses
# input_text = "My boyfriend and I are long distance. We have..."
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Load the model and tokenizer
# model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Tokenize input
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer(Input_text, return_tensors="pt", truncation=True, padding=True)
# Set the seed globally for PyTorch
seed = random.randint(0, 2**32 - 1)  # Random seed
torch.manual_seed(seed)  # For CPU operations
torch.cuda.manual_seed_all(seed)  # For GPU operations

responses = model.generate(
    inputs["input_ids"].cuda(),
    max_length=281,
    num_return_sequences=5,  # Generate multiple responses
    temperature=1.5,  # Increase temperature for randomness
    top_p=0.9,  # Use nucleus sampling (top_p)
    top_k=50,  # Sampling from top 50 tokens
    do_sample=True,  # Enable sampling
    repetition_penalty=1.5,  # Penalize repetitive words
    no_repeat_ngram_size=3,  # Prevent repeating 3-grams
).cuda()
# Decode and print the generated responses
decoded_responses = [tokenizer.decode(response, skip_special_tokens=True) for response in responses]

for idx, response in enumerate(decoded_responses):
    print(f"Response {idx+1}: {response}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response 1: Using a throwaway since I don't want this tied to my main account. Anyways, I have a friend whom I haven't talked to in a year. I recently made a new facebook, added her (which she accepted), and sent her a message. We've known each other since high school, so about 5 years now.

I asked her if she would like to meet up to catch up on things, she said she might be free this week (that was last friday). I told her just message me with a convenient time and she said sure. Now, I'm close to positive she won't respond again, which is fine, but I've been trying to mend a lot of friendships. I've 'restored' 3 or 4 now.

I'm trying to change my lifestyle, and 'old me' would have sent a couple messages on monday/tuesday, but I don't want to sound creepy or needy. I genuinely want to send a thought out apology.

I just need some help on getting it across in a decent way. Even if she just says she doesn't want to be my friend, I just want to send it as a peace of mind. I've restored 

**Now we have generated the sample we can further perform RLHF inference by assuming the feedback and for each of the response and ranking each of the response this way DPO will optimize**

In [180]:
torch.cuda.empty_cache()

## Few Model Performance Experements

In [48]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # Batch size for training on each device (e.g., GPU)
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=0.0005,
    eval_strategy="steps",
    logging_first_step=True,
    logging_steps=100,  # match results in blog post
    eval_steps=100,
    output_dir="./test",
    optim="adamw_hf",
    warmup_steps=150,
    report_to="tensorboard",
    bf16=True,
    gradient_checkpointing=gradient_checkpointing,
    num_train_epochs=3,  # Number of training epochs

    # TODO: uncomment that on the next transformers release
    # gradient_checkpointing_kwargs=gradient_checkpointing_kwargs,
)

In [49]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset = dataset_eval,
    tokenizer=tokenizer,
)
dpo_trainer.train()

/tmp/ipykernel_3687305/1875480854.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  super(DPOTrainer, self).__init__(*args, **kwargs)
/home/jupyter-st125490/.local/lib/python3.12/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/jupyter-st125490/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.042800,No log
200,0.349800,No log
300,0.306200,No log
400,0.277200,No log
500,0.257300,No log
600,0.224000,No log
700,0.193500,No log
800,0.157300,No log
900,0.155900,No log
1000,0.146900,No log


/home/jupyter-st125490/.local/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1000, training_loss=0.32084336376190187, metrics={'train_runtime': 1075.7862, 'train_samples_per_second': 3.718, 'train_steps_per_second': 0.93, 'total_flos': 1045168128000000.0, 'train_loss': 0.32084336376190187, 'epoch': 4.0})

**From above generation**

In [57]:
torch.cuda.empty_cache()

In [31]:
torch.save(model,'DPO-GPT2')

In [47]:
pad_token_id = tokenizer.pad_token_id
padding_length = 1
if padding_length > 0:
    input_ids = torch.cat([inputs['input_ids'], torch.full((inputs['input_ids'].shape[0], padding_length), pad_token_id)], dim=1)
attention_mask = torch.cat([inputs['attention_mask'], torch.zeros((inputs['attention_mask'].shape[0], padding_length), dtype=torch.long)], dim=1)


In [48]:
{'input_ids':input_ids.unsqueeze(0).cuda().size(), 
 'attention_mask':attention_mask.unsqueeze(0).cuda().size(),
 'response_1':responses[0].unsqueeze(0).cuda().size(),
'response_2':responses[1].unsqueeze(0).cuda().size(),
'preference':torch.tensor([0,1], dtype = torch.long)}

{'input_ids': torch.Size([1, 1, 281]),
 'attention_mask': torch.Size([1, 1, 281]),
 'response_1': torch.Size([1, 281]),
 'response_2': torch.Size([1, 281]),
 'preference': tensor([0, 1])}

In [49]:
dpo_trainer.compute_loss(model.to("cuda:0"),{'input_ids':input_ids[0].unsqueeze(1).cuda(), 
 'attention_mask':attention_mask[0].unsqueeze(0).cuda(),
 'response_1':responses[0].unsqueeze(0).cuda(),
'response_2':responses[1].unsqueeze(0).cuda(),
'preference':torch.tensor([0,1], dtype = torch.long).cuda()})

tensor(5.8830, device='cuda:0', grad_fn=<MeanBackward0>)

In [50]:
dpo_trainer.compute_loss(model.to("cuda:0"),{'input_ids':input_ids[0].unsqueeze(1).cuda(), 
 'attention_mask':attention_mask[0].unsqueeze(0).cuda(),
 'response_1':responses[1].unsqueeze(0).cuda(),
'response_2':responses[2].unsqueeze(0).cuda(),
'preference':torch.tensor([1,1], dtype = torch.long).cuda()})

tensor(5.8860, device='cuda:0', grad_fn=<MeanBackward0>)

In [51]:
dpo_trainer.compute_loss(model.to("cuda:0"),{'input_ids':input_ids[0].unsqueeze(1).cuda(), 
 'attention_mask':attention_mask[0].unsqueeze(0).cuda(),
 'response_1':responses[3].unsqueeze(0).cuda(),
'response_2':responses[4].unsqueeze(0).cuda(),
'preference':torch.tensor([0,0], dtype = torch.long).cuda()})

tensor(5.8841, device='cuda:0', grad_fn=<MeanBackward0>)

In [65]:
from IPython.display import display, HTML
import pandas as pd
v=pd.DataFrame(columns =['Reward loss', 'Dataset'])
v['Reward loss'] = ['6.2984', '6.2947', '6.2947']
v['Dataset'] = ['0,0','1,2', '3,4']
display(v)

,Reward loss,Dataset
0,6.2984,"0,0"
1,6.2947,"1,2"
2,6.2947,"3,4"


**Where 0 and 1 is the sentence generated**

**Observation challanges and mistakes**

* our objective was to make summarization model
* we are using feedback_suumarization dataset
* each prompt have multiple responses, each and each set of reponse have preference score 1 or 0 which is human in loop 
* our objective was to score more and better generalized model
* now at the end with the given provied time we tried to make things possible and due to shortage of time could not make changes
* i realied that GPT-2 is not suitable for summarization
* we named it text generation or where given the input the model tend to generate text as per the genrated text above